In [1]:
import sys
from skipgram import SkipGram
from positional_encoder import PositionalEncoder

from encoder import Encoder
from decoder import Decoder

from transformer import Transformer

sys.path.append("../")
from utils.preprocess import clean_text

In [2]:
D_MODEL = 128
CONTEXT_SIZE = 6
EPOCHS = 10

In [3]:
with open("../../data/art_of_war.txt") as f:
    raw_data = f.read()

cleaned_data = clean_text(raw_text=raw_data)

In [4]:
word2vec = SkipGram(d_model=D_MODEL, context_size=CONTEXT_SIZE)
word2vec.fit(corpus=cleaned_data.split(" "), epochs=5, device='cuda')

In [8]:
seq = word2vec.encode(cleaned_data.split(" "))

In [9]:
pe = PositionalEncoder(d_model=D_MODEL)
for i in range(seq.shape[0]):
    seq[i] = seq[i] + pe(i)

In [10]:
decoder_inputs = [s[:-1] for s in seq]
decoder_targets = [s[1:] for s in seq]

In [25]:
model = Transformer(num_blocks=4, d_model=D_MODEL, num_heads=4, hidden_dim=128, source_vocab_size=word2vec.vocab_size, target_vocab_size=word2vec.vocab_size, max_input_len=seq.shape[0], max_target_len=100)

In [31]:
for i in range(EPOCHS):
    model.train()